In [1]:
import numpy as np
import torch
import torch.nn as nn
import importlib as imp
np.random.seed(0)
torch.manual_seed(0)

In [12]:
import data
imp.reload(data)
manager = data.Meme_n_Caption_Data_Manager(
    FILENAME_all_image_embeddings='image_embeddings.pt',
    FILENAME_all_caption_embeddings='caption_embeddings.pt',
    FILENAME_MAP_image_index_TO_caption_indices='image_index_TO_caption_index.npy'
)
all_indices = list(range(len(manager.MAP_image_index_TO_caption_indices[0])))
np.random.shuffle(all_indices)
train_ratio, validate_ratio, test_ratio = 0.7, 0.1, 0.2 # 0.7, 0.1, 0.2
# assert train_ratio + validate_ratio + test_ratio == 1, "STOP! Train/Validate/Test split NO add to 1"
test_indices = np.asarray(all_indices[int(len(all_indices) * (1 - test_ratio)):])
train_and_validate_indices = np.asarray(all_indices[:int(len(all_indices) * (1 - test_ratio))])
# assert len(train_and_validate_indices) + len(test_indices) == len(all_indices), "STOP! T/V/T indices split wrong"
print(f"for each of the 300 images, Train & Validate uses {len(train_and_validate_indices)} captions (out of {len(all_indices)})")
# train_indices = all_indices[:int(len(all_indices) * (train_ratio))]
# validate_indices = all_indices[int(len(all_indices) * (train_ratio)):int(len(all_indices) * (1 - test_ratio))]
# assert len(train_indices) + len(validate_indices) + len(test_indices) == len(all_indices), "STOP! T/V/T indices split wrong"

for each of the 300 images, Train & Validate uses 2850 captions (out of 3000)


In [ ]:
from numpy.typing import NDArray
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import model
imp.reload(model)
imp.reload(data)

def cross_validate(m: nn.Module, hyperparameters,
        plot: bool=False):
    DOER_CV = KFold(n_splits=5, shuffle=True, random_state=0)
    for train_indicesCEPTION, validate_indicesCEPTION in DOER_CV.split(
        np.asarray(train_and_validate_indices)[:, None], train_and_validate_indices):
        train_losses = model.train(model=m, 
            dataloader_train=manager.get_dataloader(
                batch_size=hyperparameters['batch_size'], 
                RATIO_negative_samples_over_positive_samples=hyperparameters['ratio_-_samples_count_over_+_samples_count'], 
                INDICES_usable_captions=train_and_validate_indices[train_indicesCEPTION]), hyperparameters=hyperparameters)
        print(train_losses[0:10])
        if plot:
            f, ax = plt.subplots(figsize=(4, 4))
            ax.plot(train_losses)
        # break
    # inference
    test_losses, test_accuracy, aurocs, auprcs = model.test(model=m, 
        dataloader_test=manager.get_dataloader(
            batch_size=hyperparameters['batch_size'], 
            RATIO_negative_samples_over_positive_samples=hyperparameters['ratio_-_samples_count_over_+_samples_count'], 
            INDICES_usable_captions=test_indices),
        hyperparameters=hyperparameters
        )
    auroc_mean, auroc_std = np.asarray(aurocs).mean(), np.asarray(aurocs).std()
    auprc_mean, auprc_std = np.asarray(auprcs).mean(), np.asarray(auprcs).std()
    losses_mean, losses_std = np.asarray(test_losses).mean(), np.asarray(test_losses).std()
    # print(m)
    print(f"Test AUROC: {auroc_mean, auroc_std}")
    print(f"Test AUPRC: {auprc_mean, auprc_std}")
    print(f"Test Loss: {losses_mean, losses_std}")
    print(f"Test Accuracy: %{test_accuracy}")
    return m

hyperparameters = {'learning_rate': 1e-3, 'batch_size': 64, 
    'optimizer_type': torch.optim.Adam, 'ratio_-_samples_count_over_+_samples_count': 1.0,
    'loss_function': nn.BCELoss}
m = cross_validate(model.Siamese(), hyperparameters,
    plot=True)